## WPL Financial Model

#### Notes of to dos
- Need to figure out how to add solar extension period + end effects period
- will make bige ole Dictionary of dfs of depreciations
- start thinking about how to automate QC'ing - ask Barett and Rowen about what they do to QC typically
- Create functions to apply formatting
    - https://pandas.pydata.org/docs/user_guide/style.html
    - also from openpyxl.styles import Font, PatternFill, Alignment
- automate making tables in "Financial Portfolios" tab --> the ones that are not hard coded


### Import packages

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
import pathlib


# Pause future warnings for cleaner output
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Import data and supporting functions

In [2]:
# Set the path to the folder containing your Excel files
folder_path = '/Users/alomsadze/OneDrive - Charles River Associates International/Desktop/WPL/Python Version/'
model_inputs = pd.ExcelFile(folder_path + "Direct Model Inputs.xlsx")

from data_processing_functions import *
from O_and_M_functions import *


### Import Data

In [3]:
financial_inputs = model_inputs.parse("Financial Inputs", header=None)
financial_inputs_tables = read_excel_with_tables(financial_inputs)
financial_scalars_inputs = financial_inputs_tables['Scalar Inputs'].set_index('Scalar Input')
inflation_vector_df = financial_inputs_tables['Inflation Vector - Base Year 2021$']
inflation_vector = inflation_vector_df.set_index('Year')['Scalar']
CCS_inputs = model_inputs.parse("CCS", header=None)
CCS_inputs_tables = read_excel_with_tables(CCS_inputs)

In [4]:
aurora_portfolio_summary = model_inputs.parse("Portfolio Summary")
capacity_payments = model_inputs.parse("Capacity Payments")
datacenter_scenario_financials = model_inputs.parse("Datacenter Scenario Financials", header=None)
datacenter_scenario_financials_tables = read_excel_with_tables(datacenter_scenario_financials)
hydrogen_island_inputs = read_excel_with_tables(model_inputs.parse("Hydrogen Island", header=None))
AS_RT_inputs = model_inputs.parse("AS_RT Value")

In [5]:
new_capacity_additions_annual_df = remove_whitespaces_from_df(datacenter_scenario_financials_tables['New Capacity Additions Annual (MW)'].set_index('Year'))


## Revenue Requirement Calculations

In [6]:
# GLOBAL VARIABLES TO GENERALIZE
Run_ID = 'CIC'
Scenario = 'Continue_Change'
year = 2023 
aurora_portfolio_ID = 2
aurora_condition = 'ATC'
case_name = "Datacenter"

In [7]:
# years we want to calculate revenue requirement for (not in use yet)
start_year = 2023
end_year = 2072
years = np.arange(start_year, end_year + 1)

### 1. O&M Summary

#### 1a. Variable O&M

In [8]:
# Total owned resources costs (no market purchases/contracts)
total_owned_cost_yearly = []
# Market purchases cost
market_purchases_yearly = []
# Revenues from market sales
market_sales_yearly = []
# Cost of contracts
contract_costs_yearly = []
# Contract Sales
contract_sales_yearly = []
# High Load Capacity Payments = Cost of short capacity in high load scenario
capacity_payments_yearly = []

condition_mask = aurora_portfolio_summary['Condition'] == aurora_condition
run_id_mask = aurora_portfolio_summary['Run_ID'] == Run_ID
portfolio_id_mask = aurora_portfolio_summary['Portfolio_ID'] == aurora_portfolio_ID
aurora_portfolio_summary_filtered = aurora_portfolio_summary[condition_mask & run_id_mask & portfolio_id_mask]

capacity_payments_filtered = capacity_payments[(capacity_payments.Scenarios == Scenario) & (capacity_payments['Case Name'] == case_name)]

aurora_years = np.sort(aurora_portfolio_summary.Time_Period.unique())

for year in aurora_years:
    # select row for current year
    curr_year_data = aurora_portfolio_summary_filtered.loc[(aurora_portfolio_summary_filtered['Time_Period'] == year)]
    curr_year_capacity_payments = capacity_payments_filtered[year]
    # extract data
    total_owned_cost_yearly.append(curr_year_data["Resource_Cost_Total"].sum() * 1000)
    market_purchases_yearly.append(curr_year_data['Market_Purchases_Cost_Total'].sum() * 1000)
    market_sales_yearly.append(curr_year_data['Market_Sales_Cost_Total'].sum() * 1000)
    contract_costs_yearly.append(curr_year_data['Contract_Purchases_Cost_Total'].sum() * 1000)
    contract_sales_yearly.append(curr_year_data['Contract_Sales_Cost_Total'].sum() * 1000)
    capacity_payments_yearly.append(curr_year_capacity_payments.sum())
    
# Summarize
# Net market purchases = Purchases + Sales
net_market_purchases_yearly = [sum(costs) for costs in zip(market_purchases_yearly, market_sales_yearly)]
total_portfolio_cost_yearly= [sum(costs) for costs in zip(total_owned_cost_yearly, net_market_purchases_yearly,
                                                  contract_costs_yearly, contract_sales_yearly)]
total_portfolio_cost_yearly

# dictionary of lists 
VOM_portfolio_cost_dict = {'Year': aurora_years,
        'Total Portfolio Cost': total_portfolio_cost_yearly, 
        'Total Owned Cost': total_owned_cost_yearly, 
        'Net Market Purchases': net_market_purchases_yearly, 
        'Market Purchases (Energy)': market_purchases_yearly,
        'Market Sales (Energy)': market_sales_yearly, 
        'Contract Cost': contract_costs_yearly, 
        'Contract Sales': contract_sales_yearly,
        'High Load Capacity Payment': capacity_payments_yearly} 
   
VOM_portfolio_cost_df = pd.DataFrame(VOM_portfolio_cost_dict)
VOM_portfolio_cost_df.set_index('Year', inplace=True)
VOM_portfolio_cost_df = VOM_portfolio_cost_df.T
VOM_portfolio_cost_df.style.format(precision=0)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046
Total Portfolio Cost,360114373,315543248,254601854,266444572,297370895,316452912,335602806,340641207,325180211,329969430,320359316,334487855,251862719,256107863,264522406,278117395,294598145,319782793,339273961,357824755,378143000,392755512,333651893,341379343,267445956
Total Owned Cost,537460188,471145219,353665406,262398250,253670625,269277125,293761969,308243188,325586125,245003406,229553406,228581312,267606250,267590438,263890875,268308562,276112906,270473062,286528219,293996719,333906562,333948344,350943906,232417156,234832531
Net Market Purchases,-211586578,-206615415,-155103591,-45490605,-1956746,880043,-9773449,-21969156,-55561977,28918559,35088441,49422453,-66399035,-52659836,-41258141,-32482688,-24464848,5588797,11931453,51119648,31557672,54768781,-21341078,104923102,28574812
Market Purchases (Energy),27016125,16312913,15318784,37247598,57834707,64844215,63107496,53082539,48967758,86054789,90726930,100006852,42647191,49081227,54515609,60523109,64799777,82657305,91013797,120368688,116709039,133438297,130535000,197774578,175095375
Market Sales (Energy),-238602703,-222928328,-170422375,-82738203,-59791453,-63964172,-72880945,-75051695,-104529734,-57136230,-55638488,-50584398,-109046227,-101741062,-95773750,-93005797,-89264625,-77068508,-79082344,-69249039,-85151367,-78669516,-151876078,-92851477,-146520562
Contract Cost,39703293,56600320,61741785,55335277,51587680,52316910,53354586,54367176,55156062,56047465,55717469,56484090,50655504,41177262,41889672,42291520,42950086,43720934,40814289,12708388,12678766,4038387,4049065,4039085,4038613
Contract Sales,-5462529,-5586876,-5701747,-5798350,-5930664,-6021166,-1740299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
High Load Capacity Payment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 1b. Other Fixed Costs

In [9]:
# FORMAT SOME INPUT TABLES FOR EASIER DATA ACCESS
cumulative_installed_capacity_MW_df = datacenter_scenario_financials_tables['Cumulative Installed Capacity (MW)'].T
cumulative_installed_capacity_MW_df.columns = cumulative_installed_capacity_MW_df.iloc[0]
cumulative_installed_capacity_MW_df = cumulative_installed_capacity_MW_df[1:].rename_axis('Year')
cumulative_installed_capacity_MW_df = remove_whitespaces_from_df(cumulative_installed_capacity_MW_df)

FOM_2021_kw_year_df = datacenter_scenario_financials_tables['Fixed O&M ($2021/kW-yr)'].T
FOM_2021_kw_year_df.columns = FOM_2021_kw_year_df.iloc[0]
FOM_2021_kw_year_df = FOM_2021_kw_year_df[1:].rename_axis('Year')
FOM_2021_kw_year_df = remove_whitespaces_from_df(FOM_2021_kw_year_df)

AS_RT_inputs = AS_RT_inputs[AS_RT_inputs.Scenario == Scenario]
AS_RT_inputs = AS_RT_inputs.drop(columns = 'Scenario').set_index('Year')
AS_RT_inputs = remove_whitespaces_from_df(AS_RT_inputs)

FOM_years = cumulative_installed_capacity_MW_df.index.values

##### 1b.i FOM, Tax, and New Unit FOM

In [10]:
# Create an empty dictionary to store general, non-resource dependent FOM information
FOM_yearly_general_dict = {}

# Add the 'Year' key and assign the years we have for FOM as its value
FOM_yearly_general_dict['Year'] = FOM_years

# Preprocess data for calculations and extra data
transmission_upgrade_costs_df = datacenter_scenario_financials_tables['Transmission Upgrade Costs - DR Costs']
FOM_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'FOM']
FOM_yearly = FOM_yearly.loc[:, FOM_yearly.columns.isin(FOM_years)].values[0]
Transmission_Upgrade_OpEx_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'Transmission Upgrade OpEx']
Transmission_Upgrade_OpEx_yearly = Transmission_Upgrade_OpEx_yearly.loc[:, Transmission_Upgrade_OpEx_yearly.columns.isin(FOM_years)].values[0]
DSM_Costs_yearly = transmission_upgrade_costs_df[transmission_upgrade_costs_df['Category'] == 'DSM Costs']
DSM_Costs_yearly = DSM_Costs_yearly.loc[:, DSM_Costs_yearly.columns.isin(FOM_years)].values[0]
PTC_or_ITC = financial_scalars_inputs.loc['Long-term solar projects ITCs or PTCs?'].values[0]
if PTC_or_ITC == "ITC":
    tax_equity_costs_df = datacenter_scenario_financials_tables['Tax Equity Costs']
    tax_equity_costs_CA1_CA2_yearly = tax_equity_costs_df[tax_equity_costs_df.Category == 'Cash Distributions/OpEx for TE - CA1 & CA2']
    tax_equity_costs_CA1_CA2_yearly = tax_equity_costs_CA1_CA2_yearly.loc[:, tax_equity_costs_CA1_CA2_yearly.columns.isin(FOM_years)].values[0]
    tax_equity_costs_longterm_solar_yearly = tax_equity_costs_df[tax_equity_costs_df.Category == 'Cash Distributions/OpEx for TE- Long-Term Solar']
    tax_equity_costs_longterm_solar_yearly = tax_equity_costs_longterm_solar_yearly.loc[:, tax_equity_costs_longterm_solar_yearly.columns.isin(FOM_years)].values[0]
else:
    tax_equity_costs_CA1_CA2_yearly = [0] * len(FOM_years)
    tax_equity_costs_longterm_solar_yearly = [0] * len(FOM_years)

# Save data into dictionary
FOM_yearly_general_dict['FOM'] = FOM_yearly
FOM_yearly_general_dict['Transmission Upgrade OpEx'] = Transmission_Upgrade_OpEx_yearly
FOM_yearly_general_dict['DSM Costs'] = DSM_Costs_yearly
FOM_yearly_general_dict['Tax Equity Costs - CA1 & CA2'] = tax_equity_costs_CA1_CA2_yearly
FOM_yearly_general_dict['Tax Equity Costs - Long-Term Solar'] = tax_equity_costs_longterm_solar_yearly

# Create a DataFrame from the dictionary
FOM_yearly_general_df = pd.DataFrame(FOM_yearly_general_dict)

# Set 'Year' as the index and transpose the DataFrame
FOM_yearly_general_df.set_index('Year', inplace=True)
FOM_yearly_general_df = FOM_yearly_general_df.T
FOM_yearly_general_df

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
FOM,50201214.932678,59707421.540525,56192856.655525,58415753.765196,71870573.805693,68329970.659127,70350835.962421,62381445.807878,71355624.646308,43302290.186492,...,51160694.790682,51096581.517852,51041855.430041,51720035.356483,52394896.232525,52576138.344097,52330265.995222,33675490.813609,34349000.629881,26586153.615563
Transmission Upgrade OpEx,0,122425.406211,677965.17166,677965.17166,775528.177087,3763883.033307,5797956.572107,7874745.655223,9995147.309084,12160077.397676,...,15522573.486177,15522573.486177,16501402.475074,18100417.511337,20821408.098043,20821408.098043,37308402.459439,37308402.459439,40265257.322078,40265257.322078
DSM Costs,698000,923260.5,985775.5,1110848,1235920.5,1360993.0,1486065.5,1611138.0,1736210.5,1772670.9205,...,2050257.055278,2093312.453439,2137272.014961,2182154.727275,2227979.976548,2274767.556056,2322537.674733,2371310.965902,2421108.496186,2471951.774606
Tax Equity Costs - CA1 & CA2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tax Equity Costs - Long-Term Solar,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Create an empty dictionary to store yearly AS_RT costs by resource
FOM_yearly_by_resource_dict = {}

# Add the 'Year' key and assign the years we have for FOM as its value
FOM_yearly_by_resource_dict['Year'] = FOM_years

# Iterate through each resource in the columns of AS_RT_inputs
for resource in AS_RT_inputs.columns:
    # Get the yearly AS_RT values for the current resource and save to dictionary 
    yearly_FOM = get_resource_FOM(resource,
                                      cumulative_installed_capacity_MW_df, 
                                      FOM_2021_kw_year_df, 
                                      inflation_vector).values[0]
    FOM_yearly_by_resource_dict[resource + ' - FOM'] = yearly_FOM
    
###### DEAL WITH OUTLIERS ######
    
# Gas CCGT FOM is a bit more involved because we need to add in separate CSS FOM values
# NOTE - CSS FOM starts a year before we actually have capacity. TBD on what to do here.
# This is also currently incorrectly coded in the Excel model - CSS values offset by a year in the summation.
Gas_CCGT_with_CCS_FOM_yearly = get_resource_FOM('Gas CCGT with CCS',
                                   cumulative_installed_capacity_MW_df, 
                                   FOM_2021_kw_year_df, 
                                   inflation_vector)
CCS_FOM_df = CCS_inputs_tables['$ FOM'][CCS_inputs_tables['$ FOM']['Aurora ID'] == aurora_portfolio_ID].drop(columns='Aurora ID')
Gas_CCGT_FOM_df = pd.concat([CCS_FOM_df, Gas_CCGT_with_CCS_FOM_yearly])
Gas_CCGT_with_CCS_FOM_yearly = Gas_CCGT_FOM_df.sum(skipna=True).values
FOM_yearly_by_resource_dict['Gas CCGT with CCS - FOM'] = Gas_CCGT_with_CCS_FOM_yearly

# Hydrogen Island is a bit more involved because we need to use a separate input that is nominal (not pre-inflated)
hydrogen_island_capacity_yearly = cumulative_installed_capacity_MW_df['H2 Island']
hydrogen_island_capacity_yearly = hydrogen_island_capacity_yearly * 1000 # convert MWs to KWs
hydrogen_island_FOM = hydrogen_island_inputs['FOM'].set_index('Year')
hydrogen_island_FOM_yearly = hydrogen_island_FOM[Scenario]
hydrogen_island_yearly_df = pd.merge(hydrogen_island_capacity_yearly, hydrogen_island_FOM_yearly, on='Year')
hydrogen_island_yearly_df.columns = ['Capacity','FOM']
hydrogen_island_yearly_df['Total FOM'] =  hydrogen_island_yearly_df['Capacity'] * hydrogen_island_yearly_df['FOM']
hydrogen_total_FOM_yearly = hydrogen_island_yearly_df['Total FOM'] 
H2_island_FOM_yearly = hydrogen_total_FOM_yearly.to_frame().T
FOM_yearly_by_resource_dict['H2 Island - FOM'] = H2_island_FOM_yearly.values[0]

######

# Create a DataFrame from the dictionary
FOM_portfolio_cost_df = pd.DataFrame(FOM_yearly_by_resource_dict)

# Set 'Year' as the index and transpose the DataFrame
FOM_portfolio_cost_df.set_index('Year', inplace=True)
FOM_portfolio_cost_df = FOM_portfolio_cost_df.T

# Calculate the total O&M and add it as a new row in the DataFrame
new_unit_FOM_yearly_sum = FOM_portfolio_cost_df.sum(axis=0, skipna=True)
FOM_portfolio_cost_df = pd.concat([FOM_portfolio_cost_df.iloc[:0], pd.DataFrame([new_unit_FOM_yearly_sum]), FOM_portfolio_cost_df.iloc[0:]])
FOM_portfolio_cost_df.rename(index={FOM_portfolio_cost_df.index[0]: 'New Unit FOM'}, inplace=True)

FOM_portfolio_cost_df.style.format(precision=0)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
New Unit FOM,2680048,9258611,10356684,13713089,16020028,24436842,31434137,39414737,54099052,62166483,70564552,73592976,192009804,196415460,200570351,204208622,208081522,211274716,217192626,224414212,238491218,243984857,290596511,303714502,338196233,222103476
Wind - WI - FOM,0,0,0,0,0,6641556,13562057,20770290,28275289,36086337,44212980,45141453,46089423,47057301,48045504,49054460,50084604,51136380,52210244,53306660,63497116,64830555,75647997,77236605,88715895,90578929
H2 Island - FOM,0,0,0,0,0,0,0,0,0,0,0,0,0,640442,647765,655249,662898,670714,678700,686856,701280,716007,731043,746395,762070,778073
H2 Enabled RICE - FOM,0,0,0,0,0,1765704,1802784,2530883,7282270,7435198,7591337,7750755,7913521,8079705,8249379,8422615,8599490,9063308,10699518,11219457,11455066,12311181,18854574,19250520,19654781,20067531
Paired Li Ion Battery_10% Bonus ITC - FOM,0,0,0,3430823,3502870,3576430,3651535,3728218,3806510,3886447,3968062,4051392,4136471,4223337,4312027,4402579,4495034,4589429,4685807,4784209,4884678,4987256,5091988,5198920,5308097,5419567
Standalone Li Ion Battery - FOM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13366469,13647165,13933756,14226365
Paired Li Ion Battery - FOM,0,0,1260097,1286559,3065011,3129377,3195094,3262191,5709765,5829671,5952094,6077088,6204706,6335005,6468040,6603869,6742550,6884144,7028711,7176314,7327017,7480884,7637983,7798380,7962146,8129351
Paired Li Ion Battery_no ITC - FOM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Solar and Storage - WI - FOM,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6625404,8793899,15885165,16218753
CA1 - 2022 - FOM,2680048,2763130,2714776,2684669,2659303,2622613,2593939,2565623,2537659,2510044,2482773,2455841,2429243,2402974,2377032,2351410,2326106,2301114,2276430,2252051,2227972,2274759,2322529,2371302,2421099,2471942


##### 1b.ii New Unit Subhourly / Ancillary Revenue
Subhourly and A/S Revenue associated with new builds

In [12]:
# Create an empty dictionary to store yearly AS_RT costs by resource
AS_RT_yearly_by_resource_dict = {}

# Add the 'Year' key and assign the years we have for FOM as its value
AS_RT_yearly_by_resource_dict['Year'] = FOM_years

# Iterate through each resource in the columns of AS_RT_inputs
for resource in AS_RT_inputs.columns:
    # Get the yearly AS_RT values for the current resource and save to dictionary 
    yearly_AS_RT = get_resource_AS_RT(resource, cumulative_installed_capacity_MW_df, AS_RT_inputs).values[0]
    AS_RT_yearly_by_resource_dict[resource + ' - SH/AS Rev'] = yearly_AS_RT

# Create a DataFrame from the dictionary
AS_RT_portfolio_cost_df = pd.DataFrame(AS_RT_yearly_by_resource_dict)

# Set 'Year' as the index and transpose the DataFrame
AS_RT_portfolio_cost_df.set_index('Year', inplace=True)
AS_RT_portfolio_cost_df = AS_RT_portfolio_cost_df.T

# Calculate the total O&M and add it as a new row in the DataFrame
new_unit_AS_RT_yearly_sum = AS_RT_portfolio_cost_df.sum(axis=0, skipna=True)
AS_RT_portfolio_cost_df = pd.concat([AS_RT_portfolio_cost_df.iloc[:0], pd.DataFrame([new_unit_AS_RT_yearly_sum]), AS_RT_portfolio_cost_df.iloc[0:]])
AS_RT_portfolio_cost_df.rename(index={AS_RT_portfolio_cost_df.index[0]: 'New Unit Subhourly / Ancillary Revenue'}, inplace=True)

AS_RT_portfolio_cost_df.style.format(precision=0)

Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
New Unit Subhourly / Ancillary Revenue,0,0,-2231266,-8181309,-12374544,-15919547,-16715280,-18736078,-32899783,-34020600,-35141417,-36262234,-37383051,-38503868,-39265721,-40027575,-40789428,-42119753,-45804037,-47313571,-48259842,-50459897,-95648772,-99108106,-106347888,-108474846
Wind - WI - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
H2 Island - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
H2 Enabled RICE - SH/AS Rev,0,0,0,0,0,-2921793,-3094317,-4491905,-13327533,-13855587,-14383641,-14911695,-15439750,-15967804,-16381509,-16795215,-17208920,-18191097,-21527233,-22551232,-23002256,-24697159,-37786654,-38542387,-39313234,-40099499
Paired Li Ion Battery_10% Bonus ITC - SH/AS Rev,0,0,0,-5950043,-6265592,-6581141,-6896690,-7212239,-7527788,-7755774,-7983760,-8211746,-8439731,-8667717,-8801620,-8935523,-9069426,-9203329,-9337232,-9523977,-9714456,-9908745,-10106920,-10309059,-10515240,-10725545
Standalone Li Ion Battery - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-26530666,-27061279,-27602504,-28154554
Paired Li Ion Battery - SH/AS Rev,0,0,-2231266,-2231266,-5482393,-5758498,-6034604,-6310709,-11291683,-11633661,-11975640,-12317618,-12659597,-13001576,-13202430,-13403285,-13604139,-13804994,-14005848,-14285965,-14571684,-14863118,-15160380,-15463588,-15772860,-16088317
Paired Li Ion Battery_no ITC - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Solar and Storage - WI - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-5053460,-6700888,-12092526,-12334376
CA1 - 2022 - SH/AS Rev,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 1c. Combine all VOM data

In [13]:
dfs_to_stack = [VOM_portfolio_cost_df, FOM_yearly_general_df, FOM_portfolio_cost_df, AS_RT_portfolio_cost_df]
O_and_M_summary = stack_dataframes(dfs_to_stack)
# calculate total O&M and put it into summary
total_O_and_M_costs = O_and_M_summary.loc[['Total Portfolio Cost', 
                                     'High Load Capacity Payment', 
                                     'FOM', 
                                     'Transmission Upgrade OpEx', 
                                     'DSM Costs', 
                                     'Tax Equity Costs - CA1 & CA2', 
                                     'New Unit FOM', 
                                     'New Unit Subhourly / Ancillary Revenue']].sum(axis=0, skipna=True)
O_and_M_summary = pd.concat([O_and_M_summary.iloc[:0], pd.DataFrame([total_O_and_M_costs]), O_and_M_summary.iloc[0:]])
O_and_M_summary.rename(index={O_and_M_summary.index[0]: 'Total O&M Costs'}, inplace=True)
O_and_M_summary.style.format(precision=0)

df1 is missing columns: 2047


Year,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
Total O&M Costs,413693636,385554966,320583869,332180918,374898402,398425054,427956521,433187195,429466463,415350351,416142803,433348932,469677497,478973446,491737064,508651532,530623764,557650224,580343081,606928004,643818660,661952787,620560839,619340944,576329668,182951993
Total Portfolio Cost,360114373,315543248,254601854,266444572,297370895,316452912,335602806,340641207,325180211,329969430,320359316,334487855,251862719,256107863,264522406,278117395,294598145,319782793,339273961,357824755,378143000,392755512,333651893,341379343,267445956,None
Total Owned Cost,537460188,471145219,353665406,262398250,253670625,269277125,293761969,308243188,325586125,245003406,229553406,228581312,267606250,267590438,263890875,268308562,276112906,270473062,286528219,293996719,333906562,333948344,350943906,232417156,234832531,None
Net Market Purchases,-211586578,-206615415,-155103591,-45490605,-1956746,880043,-9773449,-21969156,-55561977,28918559,35088441,49422453,-66399035,-52659836,-41258141,-32482688,-24464848,5588797,11931453,51119648,31557672,54768781,-21341078,104923102,28574812,None
Market Purchases (Energy),27016125,16312913,15318784,37247598,57834707,64844215,63107496,53082539,48967758,86054789,90726930,100006852,42647191,49081227,54515609,60523109,64799777,82657305,91013797,120368688,116709039,133438297,130535000,197774578,175095375,None
Market Sales (Energy),-238602703,-222928328,-170422375,-82738203,-59791453,-63964172,-72880945,-75051695,-104529734,-57136230,-55638488,-50584398,-109046227,-101741062,-95773750,-93005797,-89264625,-77068508,-79082344,-69249039,-85151367,-78669516,-151876078,-92851477,-146520562,None
Contract Cost,39703293,56600320,61741785,55335277,51587680,52316910,53354586,54367176,55156062,56047465,55717469,56484090,50655504,41177262,41889672,42291520,42950086,43720934,40814289,12708388,12678766,4038387,4049065,4039085,4038613,None
Contract Sales,-5462529,-5586876,-5701747,-5798350,-5930664,-6021166,-1740299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
High Load Capacity Payment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None
FOM,50201215,59707422,56192857,58415754,71870574,68329971,70350836,62381446,71355625,43302290,44179983,45311958,45778741,47505085,48420670,48822430,51160695,51096582,51041855,51720035,52394896,52576138,52330266,33675491,34349001,26586154


### [appendix] Extra functionality as needed while coding

In [14]:
original_text = """
'Wind - WI - FOM': wind_FOM_yearly,
'H2 Island': H2_island_FOM_yearly,
'H2 Enabled RICE': H2_Enabled_RICE_FOM_yearly,
'Paired Li Ion Battery_10% Bonus ITC': Paired_Li_Ion_Battery_10_Bonus_ITC_FOM_yearly,
'Standalone Li Ion Battery': Standalone_Li_Ion_Battery_FOM_yearly,
'Paired Li Ion Battery': Paired_Li_Ion_Battery_FOM_yearly,
'Paired Li Ion Battery_no ITC': Paired_Li_Ion_Battery_no_ITC_FOM_yearly,
'Solar and Storage - WI': Solar_and_Storage_WI_FOM_yearly,
'CA1 - 2022': CA1_2022_FOM_yearly,
'CA1 - 2023': CA1_2023_FOM_yearly,
'CA2 - 2023': CA2_2023_FOM_yearly,
'Long-Term Solar': Long_Term_Solar_FOM_yearly,
'Flow Battery - Med 8 hrs': Flow_Battery_Med_8_hrs_FOM_yearly,
'Gas CCGT with CCS': Gas_CCGT_with_CCS_FOM_yearly,
'AGP Neenah & Sheboygan': AGP_Neenah_Sheboygan_FOM_yearly,
'AGP Riverside': AGP_Riverside_FOM_yearly
"""

# Replace "FOM" with "AS_RT"
modified_text = original_text.replace("FOM", "AS_RT")



print(modified_text)



'Wind - WI - AS_RT': wind_AS_RT_yearly,
'H2 Island': H2_island_AS_RT_yearly,
'H2 Enabled RICE': H2_Enabled_RICE_AS_RT_yearly,
'Paired Li Ion Battery_10% Bonus ITC': Paired_Li_Ion_Battery_10_Bonus_ITC_AS_RT_yearly,
'Standalone Li Ion Battery': Standalone_Li_Ion_Battery_AS_RT_yearly,
'Paired Li Ion Battery': Paired_Li_Ion_Battery_AS_RT_yearly,
'Paired Li Ion Battery_no ITC': Paired_Li_Ion_Battery_no_ITC_AS_RT_yearly,
'Solar and Storage - WI': Solar_and_Storage_WI_AS_RT_yearly,
'CA1 - 2022': CA1_2022_AS_RT_yearly,
'CA1 - 2023': CA1_2023_AS_RT_yearly,
'CA2 - 2023': CA2_2023_AS_RT_yearly,
'Long-Term Solar': Long_Term_Solar_AS_RT_yearly,
'Flow Battery - Med 8 hrs': Flow_Battery_Med_8_hrs_AS_RT_yearly,
'Gas CCGT with CCS': Gas_CCGT_with_CCS_AS_RT_yearly,
'AGP Neenah & Sheboygan': AGP_Neenah_Sheboygan_AS_RT_yearly,
'AGP Riverside': AGP_Riverside_AS_RT_yearly

